In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 6: Random Forests
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [107]:
import pandas as pd
import os

week = 5

home_path =  os.path.expanduser('~')
file_path = '/Desktop/CIS399/'

file_name = 'loan_wrangled_w'+str(week)+'.csv'

loan_table = pd.read_csv(home_path + file_path + file_name)

In [108]:
os.chdir(home_path + file_path + 'datascience_1/')
!git pull

Already up-to-date.


In [109]:
#load the lbirary from content this week

import sys
sys.path.append(home_path + '...')
from week6 import *
%who function

accuracy	 build_pred	 build_tree_iter	 caser	 compute_prediction	 compute_training	 f1	 find_best_splitter	 forest_builder	 
generate_table	 gig	 gini	 informedness	 k_fold	 predictor_case	 probabilities	 rowIndex	 seeder	 
tree_predictor	 unionset	 vote_taker	 vote_taker_union	 


In [110]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,Low,1,0,0,0,0,1,0,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0


In [111]:
loan_table.columns.values

array(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status', 'no_lam', 'filled_lam', 'pa_Rural', 'pa_Semiurban',
       'pa_Urban', 'pa_nan', 'bin_lam', 'bin_Low', 'bin_Average',
       'bin_High', 'ch_bad', 'ch_good', 'ch_nan', 'apin_binned',
       'apin_Low', 'apin_Average', 'apin_High', 'apin_nan', 'dep_0',
       'dep_1', 'dep_2', 'dep_3+'], dtype=object)

<hr>
<h1>
1: Explore forest options
</h1>
<p>
<div class=h1_cell>
<p>
Check out the results you get from forests of size 5, 11, 17. Use default depth of 2.
<p>
First, define the columns to use. I do that for you below.
</div>

In [112]:
splitter_columns = [
        #Dependents
        'dep_0', 'dep_1', 'dep_2', 'dep_3+',
        #ApplicantIncome
       'apin_Low', 'apin_High', 'apin_Average',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban',
        #LoanAmount
        'bin_Low', 'bin_Average', 'bin_High',
        #Credit_History
        'ch_bad', 'ch_good']

<hr>
<h2>
You might get different results
</h2>
<p>
<div class=h1_cell>
<p>
Since we are now in random land, your results may not match mine.
</div>

In [113]:
forest1 = forest_builder(loan_table, splitter_columns, 'Loan_Status', hypers={})
len(forest1)

5

In [114]:
loan_table['forest_1'] = loan_table.apply(lambda row: vote_taker(row, forest1), axis=1)

loan_table['forest_1_type'] = loan_table.apply(lambda row: predictor_case(row, pred='forest_1', target='Loan_Status'), axis=1)

forest1_types = loan_table['forest_1_type'].value_counts()  # returns a series

print((accuracy(forest1_types), f1(forest1_types), informedness(forest1_types)))

(0.8094462540716613, 0.8764519535374868, 0.4104956556082149)


In [115]:
forest2 = forest_builder(loan_table, splitter_columns, 'Loan_Status', hypers={'total-trees' : 11})
len(forest2)

11

In [116]:
loan_table['forest_2'] = loan_table.apply(lambda row: vote_taker(row, forest2), axis=1)

loan_table['forest_2_type'] = loan_table.apply(lambda row: predictor_case(row, pred='forest_2', target='Loan_Status'), axis=1)

forest2_types = loan_table['forest_2_type'].value_counts()  # returns a series

print((accuracy(forest2_types), f1(forest2_types), informedness(forest2_types)))

(0.6889250814332247, 0.8154589371980676, 0.005208333333333259)


In [117]:
forest3 = forest_builder(loan_table, splitter_columns, 'Loan_Status', hypers={'total-trees' : 17})
len(forest3)

17

In [118]:
loan_table['forest_3'] = loan_table.apply(lambda row: vote_taker(row, forest3), axis=1)

loan_table['forest_3_type'] = loan_table.apply(lambda row: predictor_case(row, pred='forest_3', target='Loan_Status'), axis=1)

forest3_types = loan_table['forest_3_type'].value_counts()  # returns a series
print((accuracy(forest3_types), f1(forest3_types), informedness(forest3_types)))

(0.6889250814332247, 0.8151016456921588, 0.008046998420221163)


<hr>
<h1>
2: Implement Out of Bag testing
</h1>
<p>
<div class=h1_cell>
<p>
Last module we looked at the use of K-Folding as a means to test our trees. Random Forests give us an alternative by using out of bag testing. Using notes from the content notebook this week, find a way to do prediction using the oob idea. As reminder, the set union of all the oob lists in a forest make up the testing set. If there is a row in loan_table that is not in any oob list, that row should be omitted from the test set. I am going to leave it to you to come up with an algorithm for doing oob testing. If you get totally stuck, I can supply hints.
<p>
Your goal is to be able to execute this code as is (e.g., for forest1):
<p>
<pre>
<code>
print((accuracy(forest1_oob_types), f1(forest1_oob_types), informedness(forest1_oob_types)))
</code>
</pre>
<p>
How you get to this point is up to you.
</div>

In [124]:
#Define new functions here

def unionset(forest):

    union = set()
    listofsets = []
    for tree in forest: 
        oob_set = tree['oob']
        listofsets.append(oob_set)

    #print(listofsets)

    results_union = set().union(*listofsets) ## MAGIC ALERT: https://stackoverflow.com/questions/2151517/pythonic-way-to-create-union-of-all-values-contained-in-multiple-lists
    return(results_union)

forest1_union = unionset(forest1)

print(forest1_union) ## looks like a subset to me! 

new_table = loan_table.loc[list(forest1_union)] #make new_table be the the union

print(len(loan_table)) # old has 614 enteries 
print(len(new_table)) # testing has 550 


#print(row0.index)

loan_table['forest_1_oob'] = new_table.apply(lambda row: vote_taker(row, forest1), axis=1) #notice the new_table 

loan_table['forest_1_oob_type'] = new_table.apply(lambda row: predictor_case(row, pred='forest_1_oob', target='Loan_Status'), axis=1) # new_table again 


forest1_oob_types = loan_table['forest_1_oob_type'].value_counts()  # returns a series

set([0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 68, 69, 70, 72, 73, 74, 77, 78, 81, 83, 84, 85, 86, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 227, 228, 229, 230, 231, 232, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243

In [123]:

print(forest1_oob_types.sum())  # length of testing table
print((accuracy(forest1_oob_types), f1(forest1_oob_types), informedness(forest1_oob_types)))

550
(0.8054545454545454, 0.8742655699177438, 0.39691768180140263)


<hr>
<h1>Write it out</h1>
<div class=h1_cell>

Save the table so can use it in next module.
</div>

In [18]:
import os

week = 6  # change this each week

home_path =  os.path.expanduser('~')
file_path = '/Desktop/CIS399/'

file_name = 'loan_wrangled_w'+str(week)+'.csv'

loan_table.to_csv(home_path + file_path + file_name, index=False)